In [1]:
from skimage import data
import napari
import numpy as np
from skimage import io
from pathlib import Path
import re
import os
import scipy as sp
import matplotlib.pyplot as plt
from matplotlib.backends.backend_qt5agg import FigureCanvas
from matplotlib.figure import Figure
from scipy.io import loadmat, savemat
import networkx as nx
import pickle
from scipy.ndimage import distance_transform_edt
from scipy.stats import multivariate_normal
from skimage import color, data, restoration
import vg
from pytransform3d.rotations import matrix_from_axis_angle
from skimage.feature import peak_local_max
from matplotlib.patches import Circle
from matplotlib.colors import LinearSegmentedColormap
from vispy.util.quaternion import Quaternion
import h5py
from napari_bbox import BoundingBoxLayer
import pandas as pd

In [2]:
def _rotmat(vector, points):
    """
    Rotates a 3xn array of 3D coordinates from the +z normal to an
    arbitrary new normal vector.
    """
    
    vector = vg.normalize(vector)
    axis = vg.perpendicular(vg.basis.z, vector)
    angle = vg.angle(vg.basis.z, vector, units='rad')
    
    a = np.hstack((axis, (angle,)))
    R = matrix_from_axis_angle(a)
    
    r = sp.spatial.transform.Rotation.from_matrix(R)
    rotmat = r.apply(points)
    
    return rotmat

def closest_node(node, nodes):
    nodes = np.asarray(nodes)
    dist_2 = np.sum((nodes - node)**2, axis=1)
    if np.min(dist_2)>10:
        return node
    else:
        return nodes[np.argmin(dist_2)]

In [3]:
xls = pd.ExcelFile('TBI_STIM_metalog_local.xlsx')
xls2 = pd.ExcelFile('../TBI_monai_UNET/p3_metalog.xlsx')
df = {}
for sheet_name in xls.sheet_names:
    df[sheet_name] = xls.parse(sheet_name)
for sheet_name in xls2.sheet_names:
    df[sheet_name] = xls2.parse(sheet_name)

In [4]:
exclude = ['XYZres103',
           'XYZres104',
           'XYZres105',
           'XYZres105',
           'XYZres107',
           'XYZres133',
           'XYZres134',
           'XYZres135',
           'XYZres136',
           'XYZres137',
           'XYZres138',
           'XYZres139',
           'XYZres140',
           'XYZres183',
           'XYZres196',
           'XYZres197',
           'XYZres260',
           'XYZres288',
           'XYZres343',
           'XYZres340',
           'XYZres341',
           'XYZres250',
           'XYZres297',
           'XYZres295',
           'XYZres457',
           'XYZres455']
dic = {}
for key in df.keys():
    if '3D' in key and ('vbm01' not in key and 'vbm02' not in key and 'SHAM7_3D' not in key and 'TBI45_3D' not in key and 'TBI11_3D' not in key and 'TBI65_3D' not in key and 'TBI22_3D' not in key and 'TBI28_3D' not in key and 'TBI40_3D' not in key and 'TBI51_3D' not in key and 'TBI70_3D' not in key):
        if 'vbm' not in key:
            addition = re.sub('C57','',re.sub('TBI','',re.sub('SHAM','',re.sub('_3D','',key))))
        else:
            addition = ''
        df[key] = df[key][~df[key][df[key].columns[1]].isin(exclude)]
        scans = np.array(df[key][df[key].columns[1]])
        scans = [x for x in scans if 'res' in str(x)]
        bottoms_1 = df[key][df[key][df[key].columns[3]] == 500]
        bottoms_2 = df[key][df[key][df[key].columns[2]] == 500]
        bottoms = pd.concat((bottoms_1,bottoms_2))
        bottoms = np.array(bottoms[bottoms.columns[1]])
        bottoms = [addition + '-' + x for x in bottoms]
        bottoms = [x for x in bottoms if 'res' in x]
        tops_1 = df[key][df[key][df[key].columns[3]] == 0]
        tops_2 = df[key][df[key][df[key].columns[2]] == 0]
        tops = pd.concat((tops_1,tops_2))
        tops = np.array(tops[tops.columns[1]])
        tops = [addition + '-' + x for x in tops]
        tops = [x for x in tops if 'res' in x]
        if len(tops) > 1:
            dic[tops[0]] = list(tops[1:])
        elif len(tops) == 1:
            dic[tops[0]] = tops
        if len(bottoms) > 1:
            dic[bottoms[0]] = list(bottoms[1:])
        elif len(bottoms) == 1:
            dic[bottoms[0]] = bottoms
dic_2 = {'45-XYZres290':['45-XYZres296'],
         '45-XYZres297':['45-XYZres295'],
         '45-XYZres294':['45-XYZres298'],
         '45-XYZres288':['45-XYZres300'],
         '11-XYZres95':['11-XYZres98','11-XYZres102'],
         '11-XYZres92':[],
         '11-XYZres93':['11-XYZres93','11-XYZres96','11-XYZres97','11-XYZres100','11-XYZres101'],
         '11-XYZres91':['11-XYZres94','11-XYZres99'],
         '22-XYZres164':['22-XYZres165','22-XYZres168','22-XYZres169'],
         '22-XYZres160':['22-XYZres161'],
         '22-XYZres163':['22-XYZres166','22-XYZres167','22-XYZres170'],
         '22-XYZres159':['22-XYZres162'],
         '28-XYZres184':['28-XYZres185'],
         '28-XYZres188':['28-XYZres189','28-XYZres193','28-XYZres194'],
         '28-XYZres186':['28-XYZres187','28-XYZres188','28-XYZres190','28-XYZres191','28-XYZres192','28-XYZres195'],
         '28-XYZres183':[],
         '40-XYZres248':['40-XYZres249'],
         '40-XYZres245':['40-XYZres252'],
         '40-XYZres244':[],
         '40-XYZres243':['40-XYZres246','40-XYZres247','40-XYZres251'],
         '51-XYZres297':['51-XYZres298','51-XYZres302','51-XYZres305','51-XYZres301'],
         '51-XYZres296':['51-XYZres306'],
         '51-XYZres299':['51-XYZres300','51-XYZres303','51-XYZres304'],
         '65-XYZres397':['65-XYZres398','65-XYZres401','65-XYZres402','65-XYZres405','65-XYZres408','65-XYZres409'],
         '65-XYZres396':['65-XYZres399','65-XYZres400'],
         '65-XYZres403':[],
         '70-XYZres420':['70-XYZres421','70-XYZres422'],
         '70-XYZres419':['70-XYZres416','70-XYZres413','70-XYZres410','70-XYZres412','70-XYZres416'],
         '70-XYZres414':['70-XYZres417'],
         '70-XYZres411':['70-XYZres418','70-XYZres415'],
         '-XYZres007':[]
        }
dic.update(dic_2) 
len(list(dic.keys()))

119

In [5]:
#dic.items()

In [6]:
res = {}
for k, vs in dic.items():
    #print(k)
    #print(len(vs))
    if len(vs) > 0:
        res[k] = len(vs)

print(res)


{'7-XYZres54': 7, '7-XYZres56': 7, '12-XYZres86': 4, '12-XYZres88': 4, '9-XYZres70': 7, '9-XYZres72': 7, '14-XYZres108': 3, '14-XYZres106': 3, '17-XYZres116': 5, '17-XYZres118': 3, '18-XYZres126': 3, '18-XYZres128': 3, '20-XYZres145': 4, '20-XYZres149': 4, '23-XYZres152': 3, '23-XYZres158': 3, '26-XYZres172': 5, '26-XYZres174': 5, '32-XYZres199': 5, '32-XYZres201': 5, '35-XYZres211': 6, '35-XYZres213': 5, '33-XYZres228': 2, '33-XYZres223': 1, '38-XYZres238': 4, '38-XYZres240': 4, '4-XYZres254': 4, '4-XYZres256': 3, '6-XYZres264': 7, '6-XYZres266': 5, '43-XYZres254': 3, '43-XYZres253': 1, '53-XYZres334': 4, '53-XYZres332': 4, '56-XYZres342': 2, '56-XYZres339': 1, '55-XYZres349': 3, '55-XYZres351': 2, '64-XYZres347': 6, '64-XYZres353': 6, '63-XYZres362': 4, '63-XYZres363': 3, '61-XYZres375': 6, '61-XYZres373': 7, '48-XYZres302': 6, '48-XYZres308': 6, '49-XYZres316': 6, '49-XYZres322': 6, '67-XYZres387': 6, '67-XYZres385': 6, '73-XYZres423': 6, '73-XYZres430': 1, '68-XYZres424': 2, '68-XY

In [7]:
directory_seg = Path('matt_raw_warped_single_upsampled_seg')
images = list(directory_seg.glob('*_0001_warped_seg.npy'))

images = sorted([x.as_posix() for x in images])

images = [x for x in images if os.path.exists(re.sub('_0001','',x))]

images = [x for x in images if any(y in x for y in list(dic.keys()))]
print(len(images))

key = '65-XYZres397'
seg = [x for x in images if key in x][0]
seg_files = [seg] + [re.sub(key,x,seg) for x in dic[key]]
seg_files = sorted(seg_files)
seg_files

111


['matt_raw_warped_single_upsampled_seg/20211101_65-XYZres397_0001_warped_seg.npy',
 'matt_raw_warped_single_upsampled_seg/20211101_65-XYZres398_0001_warped_seg.npy',
 'matt_raw_warped_single_upsampled_seg/20211101_65-XYZres401_0001_warped_seg.npy',
 'matt_raw_warped_single_upsampled_seg/20211101_65-XYZres402_0001_warped_seg.npy',
 'matt_raw_warped_single_upsampled_seg/20211101_65-XYZres405_0001_warped_seg.npy',
 'matt_raw_warped_single_upsampled_seg/20211101_65-XYZres408_0001_warped_seg.npy',
 'matt_raw_warped_single_upsampled_seg/20211101_65-XYZres409_0001_warped_seg.npy']

In [8]:
viewer = napari.Viewer()

In [9]:
for file in seg_files:
    img_file = re.sub('matt_raw_warped_single_upsampled_seg','matt_raw_warped_single',re.sub('_0001_warped_seg.npy','_warped.tif',file))
    img = io.imread(img_file)
    img_ch2 = sp.ndimage.zoom(np.swapaxes(img[:,1,:,:],0,2),(1,1,2.645833333))
    img = sp.ndimage.zoom(np.swapaxes(img[:,0,:,:],0,2),(1,1,2.645833333))
    seg = np.load(file)
    img = np.concatenate((np.expand_dims(img,0),np.expand_dims(img_ch2,0)),axis=0)
    viewer = napari.Viewer()

    viewer.add_image(
        img,
        channel_axis = 0,
        name = ["vessels", "neurons"],
        colormap = ['red','green'],
        scale=(2.64, 0.99, 0.99),
        rgb = False,
        depiction = 'volume',
        interpolation3d='bicubic'
    )
    viewer.scale_bar.visible = True
    viewer.scale_bar.unit = "um"
    viewer.theme = 'light'
    viewer.dims.ndisplay = 3
    
    
    
    
    
    break
    
    
    
    
    

In [41]:
img = np.concatenate((np.expand_dims(img,0),np.expand_dims(img_ch2,0)),axis=0)

(2, 512, 512, 254)

In [10]:
viewer.dims.ndisplay = 3

In [39]:
np.expand_dims(img_ch2,0)

(1, 512, 512, 254)